In [1]:
import uuid
import shapefile
import json
import pandas as pd
from shapely.geometry import shape, Point
from pyproj import Proj, transform

from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

In [2]:
# Instantiates a client
bigquery_client = bigquery.Client(project="taxianalysis-176617")

In [4]:
sf = shapefile.Reader("taxi_zones/taxi_zones")
shapes = sf.shapes()

In [5]:
NEWARK_ID = 1
LAGUARDIA_ID = 138
JFK_ID = 132

In [6]:
def get_bounds(shape):
    in_proj = Proj(init='epsg:2263', preserve_units = True)
    out_proj = Proj(init='epsg:4326')
    x1, y1 = shape.bbox[0], shape.bbox[1]
    x2, y2 = shape.bbox[2], shape.bbox[3]
    new_x1, new_y1 = transform(in_proj, out_proj, x1, y1)
    new_x2, new_y2 = transform(in_proj, out_proj, x2, y2)

    return [new_x1, new_y1, new_x2, new_y2]

In [7]:
newark_bounds = get_bounds(shapes[NEWARK_ID - 1])
laguardia_bounds = get_bounds(shapes[LAGUARDIA_ID - 1])
jfk_bounds = get_bounds(shapes[JFK_ID - 1])

In [220]:
def check_bounds(shapefile, coords):
    in_proj = Proj(init='epsg:4326')
    out_proj= Proj(init='epsg:2263', preserve_units = True)
    
    point = Point(transform(in_proj, out_proj, coords[0], coords[1]))
    polygon = shape(shapefile)
    return polygon.contains(point)

In [221]:
check_bounds(shapes[0], [-74.182861328125, 40.6878776550293])

True

In [212]:
def export_data_helper(client, table, destinationUri):
    # Export data to Google Storage
    job_name = str(uuid.uuid4())
    print job_name
    job = client.extract_table_to_storage(job_name, destination_table, destinationUri)

    job.begin()
    job.result()  # Wait for job to complete

    print('Exported {} to {}'.format(destination_table, destinationUri))

In [66]:
def export_data(client, query_bounds, dataset, destinationUri):
    
    # Query string
    query_string = """
    SELECT
        *
    FROM [bigquery-public-data:new_york.{}]
    WHERE
        /* Return values between a pair of */
        /* latitude and longitude coordinates */
        dropoff_latitude > {} AND
        dropoff_latitude < {} AND
        dropoff_longitude > {} AND
        dropoff_longitude < {}
    """.format(dataset, query_bounds[1], query_bounds[3], query_bounds[0], query_bounds[2])
    
    query_job = client.run_async_query(str(uuid.uuid4()), query_string)
    query_job.begin()
    query_job.result()
    
    destination_table = query_job.destination
    destination_table.reload()
    
    # Export data to Google Storage
    export_job_name = str(uuid.uuid4())
    export_job = client.extract_table_to_storage(export_job_name, destination_table, destinationUri)

    export_job.begin()
    export_job.result()  # Wait for job to complete

    print('Exported {} to {}'.format(destination_table, destinationUri))

In [67]:
export_data(bigquery_client, laguardia_bounds, 'tlc_yellow_trips_2016', 'gs://farallon-taxi-analysis/tlc-yellow-laguardia-dropoffs-2016.json')

Exported <google.cloud.bigquery.table.Table object at 0x115c39a50> to gs://farallon-taxi-analysis/tlc-yellow-laguardia-dropoffs-2016.json


In [62]:
newark_coordinates = {
    'terminal_a': [40.687794, -74.182307],
    'terminal_b': [40.690627, -74.177544],
    'terminal_c': [40.695558, -74.178063]
}

laguardia_coordinates = {
    'terminal_a': [40.772375, -73.885976],
    'terminal_b': [40.774444, -73.872006],
    'terminal_c': [40.770674, -73.865307],
    'terminal_d': [40.768628, -73.862134]
}

jfk_coordinates = {
    'terminal_1': [40.643325, -73.789939],
    'terminal_2': [40.641595, -73.788767],
    'terminal_4': [40.644193, -73.782554],
    'terminal_5': [40.645807, -73.776774],
    'terminal_7': [40.648798, -73.782922],
    'terminal_8': [40.646934, -73.789874]
}

In [79]:
newark_dropoffs = pd.DataFrame.from_csv('./2016/tlc-yellow-newark-dropoffs-2016.csv')

In [80]:
newark_dropoffs

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,total_amount
vendor_id,,,,,,,,,,,,,,,,,,
2,2016-02-04 07:02:49 UTC,2016-02-04 07:36:20 UTC,1,20.18,-73.985031,40.744957,3,N,-74.176781,40.694359,1,72.0,0.0,0.0,17.96,17.50,0.3,107.76
2,2016-02-24 18:39:41 UTC,2016-02-24 19:22:46 UTC,2,17.05,-74.000862,40.757599,3,N,-74.177322,40.690861,2,67.5,1.0,0.0,0.00,12.50,0.3,81.30
1,2016-01-02 05:46:50 UTC,2016-01-02 05:47:44 UTC,2,0.00,-74.182770,40.687920,5,N,-74.182770,40.687920,3,70.0,0.0,0.0,0.00,0.00,0.3,70.30
2,2016-05-16 13:06:00 UTC,2016-05-16 14:01:38 UTC,5,18.70,-73.979187,40.765572,3,N,-74.176971,40.691620,2,79.0,0.0,0.0,0.00,10.50,0.3,89.80
2,2016-05-14 14:07:36 UTC,2016-05-14 16:21:30 UTC,6,29.18,-73.871269,40.773991,3,N,-74.177307,40.694962,1,133.5,0.0,0.0,0.00,24.04,0.3,157.84
2,2016-04-28 15:12:43 UTC,2016-04-28 16:04:14 UTC,1,17.83,-73.984291,40.744289,3,N,-74.177292,40.695068,2,72.5,0.0,0.0,0.00,15.50,0.3,88.30
2,2016-05-23 11:02:36 UTC,2016-05-23 11:31:41 UTC,1,17.18,-73.989319,40.762955,3,N,-74.177711,40.695457,1,63.5,0.0,0.0,14.50,15.50,0.3,93.80
2,2016-05-09 06:31:03 UTC,2016-05-09 06:52:32 UTC,1,13.60,-74.014954,40.716412,3,N,-74.176697,40.693298,2,54.5,0.0,0.0,0.00,12.50,0.3,67.30
1,2016-04-10 09:28:11 UTC,2016-04-10 09:56:32 UTC,2,17.80,-73.972458,40.755634,3,N,-74.177246,40.695103,2,66.0,0.0,0.0,0.00,10.50,0.3,76.80
